# Wind Farm Analysis

In [2]:
# Importing libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [10]:
# Looking at Data


df = pd.read_csv('data/hly275/hly275.csv', skiprows = 17)

print(df.head())
df.info()
df.describe()






C:\Users\Marcella\AppData\Local\Temp\ipykernel_3204\498861972.py:4: DtypeWarning: Columns (2,4,6,7,8,9,10,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/hly275/hly275.csv', skiprows = 17)


                date  ind rain  ind.1 temp  ind.2 wetb dewpt vappr rhum msl  \
0  13-aug-2003 01:00   -1           4           4                             
1  13-aug-2003 02:00   -1           4           4                             
2  13-aug-2003 03:00   -1           4           4                             
3  13-aug-2003 04:00   -1           4           4                             
4  13-aug-2003 05:00   -1           4           4                             

   ind.3 wdsp  ind.4 wddir  
0      7           7        
1      7           7        
2      7           7        
3      7           7        
4      7           7        
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186744 entries, 0 to 186743
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    186744 non-null  object
 1   ind     186744 non-null  int64 
 2   rain    186744 non-null  object
 3   ind.1   186744 non-null  int64 
 4   temp    18674

,ind,ind.1,ind.2,ind.3,ind.4
count,186744.000000,186744.000000,186744.000000,186744.000000,186744.000000
mean,-0.006640,0.026560,0.028445,2.034700,2.034566
std,0.081216,0.324864,0.336114,0.415086,0.414290
min,-1.000000,0.000000,0.000000,2.000000,2.000000
25%,0.000000,0.000000,0.000000,2.000000,2.000000
50%,0.000000,0.000000,0.000000,2.000000,2.000000
75%,0.000000,0.000000,0.000000,2.000000,2.000000
max,0.000000,4.000000,4.000000,7.000000,7.000000
